In [1]:
import os
import numpy as np

import matplotlib.pyplot as plt

from lvmdrp.functions import imageMethod, rssMethod

In [2]:
# define path to calibration frames:
data_path = "_data-lvm"
CHANNEL = "z1"
cont_name = f"sdR-s-{CHANNEL}-00001614"
arc_name = f"sdR-s-{CHANNEL}-00001563"
# continuum frame
# bias
# dark
# flat
# fiberflat
# target frame
sim_path = "_sim"
sim_name = "LVM_Field_large_linear_full_9000_realization_2d"
cal_name = "LVM_Field_large_linear_full_9000_realization_rss"


# wavelength range
CHANNEL_WL = {"b1": (3600, 5930), "r1": (5660, 7720), "z1": (7470, 9800)}
wli, wlf = CHANNEL_WL["z1"]

In [3]:
_, flags = imageMethod.LACosmic_drp(
    image=f"{sim_path}/{sim_name}.fits",
    out_image=f"{sim_path}/{sim_name}.cosmic.fits",
    increase_radius=1, flim="1.3", parallel='5'
)
print(f"LACosmic_drp: {flags}")
_, flags = imageMethod.subtractStraylight_drp(
    image=f"{sim_path}/{sim_name}.cosmic.fits",
    trace=f"{sim_path}/{cal_name}.trc.fits",
    stray_image=f"{sim_path}/{sim_name}.back.fits",
    clean_image=f"{sim_path}/{sim_name}.stray.fits",
    aperture=40, poly_cross=2, smooth_gauss=30
)
print(f"subtractStraylight_drp: {flags}")
_, flags = imageMethod.extractSpec_drp(
    image=f"{sim_path}/{sim_name}.stray.fits",
    trace=f"{sim_path}/{cal_name}.trc.fits",
    out_rss=f"{sim_path}/{sim_name}.ms.fits",
    fwhm=f"{sim_path}/{cal_name}.fwhm.fits",
    method="optimal", parallel="5"
)
print(f"extractSpec_drp: {flags}")


LACosmic_drp: 1
subtractStraylight_drp: 1


In [ ]:
wave, fwhm, flags = rssMethod.detWaveSolution_drp(
    arc_rss=f"{sim_path}/{sim_name}.ms.fits",
    disp_rss=f"{sim_path}/{sim_name}.disp.fits",
    res_rss=f"{sim_path}/{sim_name}.res.fits",
    ref_line_file=f"{data_path}/lvm-neon_nist_{CHANNEL}.txt",
    aperture="7", poly_fwhm="-1,-1", poly_dispersion="-4", rel_flux_limits="0.2,2", flux_min="100.0", verbose="-1"
)
print(f"detWaveSolution_drp: {flags}")
_, flags = rssMethod.createPixTable_drp(
    rss_in=f"{sim_path}/{sim_name}.ms.fits",
    rss_out=f"{sim_path}/{sim_name}.rss.fits",
    arc_wave=f"{sim_path}/{sim_name}.disp.fits",
    arc_fwhm=f"{sim_path}/{sim_name}.res.fits",
    cropping=''
)
print(f"createPixTable_drp: {flags}")
_, flags = rssMethod.resampleWave_drp(
    rss_in=f"{sim_path}/{sim_name}.rss.fits",
    rss_out=f"{sim_path}/{sim_name}.disp_cor.fits",
    start_wave=wli, end_wave=wlf, disp_pix="1.0", err_sim="0"
)
print(f"resampleWave_drp: {flags}")


/home/mejia/.local/bin/miniconda3/envs/lvmdrp/lib/python3.8/site-packages/scipy/optimize/minpack.py:476: RuntimeWarning: Number of calls to function has reached maxfev = 9999.
  warnings.warn(errors[info][0], RuntimeWarning)


Start measuring arc lines...
detWaveSolution_drp: 1
createPixTable_drp: 1
resampleWave_drp: 1
